In [1]:
'''
Example: Unsupervised learning with the ECM algorithm.
Train data is often hard to collect in record linkage or data matching
problems. The Expectation-Conditional Maximisation (ECM) algorithm is the most
well known algorithm for unsupervised data matching. The algorithm preforms
relatively well compared to supervised methods.
'''

'\nExample: Unsupervised learning with the ECM algorithm.\nTrain data is often hard to collect in record linkage or data matching\nproblems. The Expectation-Conditional Maximisation (ECM) algorithm is the most\nwell known algorithm for unsupervised data matching. The algorithm preforms\nrelatively well compared to supervised methods.\n'

In [3]:
from __future__ import print_function

import numpy as np

import recordlinkage as rl
from recordlinkage.index import Block
from recordlinkage.compare import Exact, String

In [4]:
from recordlinkage.datasets import load_febrl3

'''
This data set contains 5000 records (2000 originals and 3000 duplicates), 
with a maximum of 5 duplicates based on one original record (and a Zipf distribution of duplicate records). 
Distribution of duplicates: 
    168 originals records have 5 duplicate records 
    161 originals records have 4 duplicate records 
    212 originals records have 3 duplicate records 
    256 originals records have 2 duplicate records 
    368 originals records have 1 duplicate record 1
    835 originals records have no duplicate record   
'''

'\nThis data set contains 5000 records (2000 originals and 3000 duplicates), \nwith a maximum of 5 duplicates based on one original record (and a Zipf distribution of duplicate records). \nDistribution of duplicates: \n    168 originals records have 5 duplicate records \n    161 originals records have 4 duplicate records \n    212 originals records have 3 duplicate records \n    256 originals records have 2 duplicate records \n    368 originals records have 1 duplicate record 1\n    835 originals records have no duplicate record   \n'

In [5]:
# set logging
rl.logging.set_verbosity(rl.logging.INFO)

In [6]:
# load dataset
print('Loading data...')
dfA, true_links = load_febrl3(return_links=True)
print(len(dfA), 'records in dataset A')
print(len(true_links), 'links in dataset A')

Loading data...
5000 records in dataset A
6538 links in dataset A


In [7]:
# start indexing
print('Build index...')
indexer = rl.Index()
indexer.add(Block('given_name'))
indexer.add(Block('surname'))
indexer.add(Block('soc_sec_id'))
candidate_links = indexer.index(dfA)
print(len(candidate_links), 'candidate links in dataset A')

Build index...
INFO:recordlinkage:indexing - initialize Index class
INFO:recordlinkage:indexing [1/?] - time: 0.41s - pairs: 75984/12497500 - rr: 0.99392
INFO:recordlinkage:indexing [1/?] - time: 0.41s - pairs_total: 75984/12497500 - rr_total: 0.99392
75984 candidate links in dataset A


In [8]:
# start comparing
print('Start comparing...')
comparer = rl.Compare()
comparer.add(Exact('given_name', 'given_name', label='given_name'))
comparer.add(String('surname', 'surname', method='jarowinkler',
                    threshold=0.85, label='surname'))
comparer.add(Exact('date_of_birth', 'date_of_birth', label='date_of_birth'))
comparer.add(Exact('suburb', 'suburb', label='suburb'))
comparer.add(Exact('state', 'state', label='state'))
comparer.add(String('address_1', 'address_1', threshold=0.85,
                    label='address_1'))
comparer.add(String('address_2', 'address_2', threshold=0.85,
                    label='address_2'))
features = comparer.compute(candidate_links, dfA)

print('feature shape', features.shape)

Start comparing...
INFO:recordlinkage:comparing - initialize Compare class
INFO:recordlinkage:comparing [1/?] - time: 1.54s - pairs: 75984
INFO:recordlinkage:comparing [1/?] - time: 1.54s - pairs_total: 75984
feature shape (75984, 7)


In [9]:
# Initialise the Expectation-Conditional Maximisation classifier.
cl = rl.ECMClassifier()
cl.fit(features)

INFO:recordlinkage:Classification - start training ECMClassifier
INFO:recordlinkage:Classification - training computation time: ~0.24s


INFO:recordlinkage:Classification - training computation time: ~0.24s


In [10]:
# Print the parameters that are trained (m, u and p). Note that the estimates
# are very good.
print("p probability P(Match):", cl.p)
print("m probabilities P(x_i=1|Match):", cl.m_probs)
print("u probabilities P(x_i=1|Non-Match):", cl.u_probs)
print("log m probabilities P(x_i=1|Match):", cl.log_m_probs)
print("log u probabilities P(x_i=1|Non-Match):", cl.log_u_probs)
print("log weights of features:", cl.log_weights)
print("weights of features:", cl.weights)

p probability P(Match): 0.08226728962662422
m probabilities P(x_i=1|Match): {'given_name': {0.0: 0.42703326870150254, 1.0: 0.5729667312984985}, 'surname': {0.0: 0.20028422600846463, 1.0: 0.7997157739915358}, 'date_of_birth': {0.0: 0.13027666124697934, 1.0: 0.8697233387530211}, 'suburb': {0.0: 0.3747709210433985, 1.0: 0.6252290789566018}, 'state': {0.0: 0.08309989904321347, 1.0: 0.9169001009567865}, 'address_1': {0.0: 0.18651502984876472, 1.0: 0.8134849701512359}, 'address_2': {0.0: 0.4123905527893048, 1.0: 0.5876094472106957}}
u probabilities P(x_i=1|Non-Match): {'given_name': {0.0: 0.48097192025547814, 1.0: 0.519028079744521}, 'surname': {0.0: 0.5161834796682475, 1.0: 0.48381652033175154}, 'date_of_birth': {0.0: 0.9999374560215991, 1.0: 6.254397840004412e-05}, 'suburb': {0.0: 0.9987567659389842, 1.0: 0.0012432340610159373}, 'state': {0.0: 0.799528787064401, 1.0: 0.20047121293559916}, 'address_1': {0.0: 0.9993129896522365, 1.0: 0.0006870103477637243}, 'address_2': {0.0: 0.9994428079221

In [11]:
# evaluate the model
links = cl.predict(features)
print("Predicted number of links:", len(links))

INFO:recordlinkage:Classification - predict matches and non-matches


INFO:recordlinkage:Classification - predict matches and non-matches


Predicted number of links: 6237


In [17]:
cm = rl.confusion_matrix(true_links, links, len(candidate_links))
print("Confusion matrix:\n", cm)

Confusion matrix:
 [[ 6232   306]
 [    5 69441]]


In [18]:
# compute the F-score for this classification
fscore = rl.fscore(cm)
print('fscore', fscore)
recall = rl.recall(true_links, links)
print('recall', recall)
precision = rl.precision(true_links, links)
print('precision', precision)

fscore 0.975655577299413
recall 0.9531966962373815
precision 0.9991983325316659


In [19]:
# Predict the match probability for each pair in the dataset.
probs = cl.prob(features)
print(probs)

INFO:recordlinkage:Classification - compute probabilities


INFO:recordlinkage:Classification - compute probabilities


rec_id_1     rec_id_2      
rec-0-org    rec-638-dup-0     0.000015
             rec-638-dup-2     0.000015
             rec-638-org       0.000015
rec-1-org    rec-1883-dup-0    0.000661
             rec-1883-dup-1    0.000661
                                 ...   
rec-999-org  rec-668-dup-1     0.000015
             rec-668-dup-2     0.000015
             rec-668-org       0.000015
             rec-679-dup-1     0.000051
             rec-679-org       0.000051
Length: 75984, dtype: float64
